In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import time
import pprint
import itertools
import pickle
import sklearn
import dask
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
# Distances
import scipy.spatial.distance as distance
# KL
from scipy.stats import entropy
# Normalize
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Pairwise distances
from sklearn.metrics.pairwise import pairwise_distances
import h5py
import dask
import dask.dataframe as dd
import dask.array as da

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Direct citation pairs

### 1. Create sample
- Citing patent granted within 10 years of cited (after clean: 9666374 pairs)
- Not from the same assignee (after clean: 8070579 pairs)

In [4]:
# %time cit = dd.read_parquet("RawData/Cleaned/cit_0628.parq")
# ip = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "gyear"])
# dup_pats = pd.read_pickle("RawData/Cleaned/duplicate_pattext_0712.pkl").tolist()

# # Remove dup pats
# cit = cit[~(cit["citing"].isin(dup_pats)) & ~(cit["cited"].isin(dup_pats))]
# ip = ip.loc[~(ip["patent"].isin(dup_pats))]

# # Merge citation with grant year
# %time cit = cit.merge(ip, how = "left", left_on = "citing", right_on = "patent").rename(columns={"gyear":"citing_gyear"})
# %time cit = cit.merge(ip, how = "left", left_on = "cited", right_on = "patent").rename(columns={"gyear":"cited_gyear"})
# cit["year_diff"] = cit["citing_gyear"]-cit["cited_gyear"]
# print(len(cit))
# cit = cit[(cit["year_diff"]<=10)]
print(len(cit))
cit = cit[(cit["citing_gyear"]>=1975) & (cit["cited_gyear"]>=1975)]
print(len(cit))

9666374
9666374


In [5]:
# Convert to dataframe
%time cit = cit[["citing", "cited"]].compute()

CPU times: user 2min 8s, sys: 19.9 s, total: 2min 28s
Wall time: 1min 2s


In [7]:
fastparquet.write("DataStore/2018-07-P2/citation_pairs_samp_0716.parq", cit, compression="GZIP")

In [9]:
# asgs = pickle.load(open("RawData/Cleaned/patent_assignee_dict_0628.pkl", "rb"))
# Check that target and other do not have same assignee
%time asg_match = (set(asgs.get(tp, [])).intersection(asgs.get(op, [])) for tp, op in zip(cit["citing"], cit["cited"]))
%time asg_match = [len(i) for i in asg_match]

%time cit["asg_match"] = [True if i >=1 else False for i in asg_match]
print(len(cit))
cit = cit.loc[cit["asg_match"] == False]
print(len(cit))

CPU times: user 1.3 s, sys: 2.87 s, total: 4.16 s
Wall time: 4.07 s
CPU times: user 28.7 s, sys: 69.8 ms, total: 28.8 s
Wall time: 28.4 s
9666374
8070579


In [11]:
fastparquet.write("DataStore/2018-07-P2/citation_pairs_samp_0716.parq", cit[["cited", "citing"]], compression="GZIP")

### 2. Get similarity

In [3]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)
logger.addHandler(logging.FileHandler('Logs/direct_cite_sim_{0}.log'.format(datetime.datetime.now().\
                                                            strftime("%Y-%m-%d"), 'a')))
print = logging.info
print('good day to you madam fiona')
print('started')
print(datetime.datetime.now())

INFO:root:good day to you madam fiona
INFO:root:started
INFO:root:2018-07-16 13:41:39.037296


In [4]:
import scipy.spatial.distance as distance
dms = ["ldavecs", "docvecs"]
# dms = ["ldavecs"]
res = {}
for dm in dms:
    print("Loading matrix and dict")
    print(datetime.datetime.now())
    
    pat_dict = fastparquet.ParquetFile("RawData/Cleaned/patabs7615_us_no_dup.parq").to_pandas(["patent"])["patent"].tolist()
    pat_dict = dict(zip(pat_dict, range(len(pat_dict))))
    # Store as dask array
    if dm == "ldavecs":
        ncols = 60
    else:
        ncols = 100
    pm = dd.read_parquet("DataStore/2018-07-P2/ML/{0}_pats_0712.parq".format(dm)).values.compute()
    pm = da.from_array(pm, chunks=(10000,ncols))

    l3 = fastparquet.ParquetFile("DataStore/2018-07-P2/citation_pairs_samp_0716.parq").to_pandas()\
    .rename(columns={"cited":"tp", "citing":"op"})

    # Remove missing values
    print(len(l3))
    l3 = l3.loc[l3["tp"].isin(pat_dict.keys()) & l3["op"].isin(pat_dict.keys())]
    print(len(l3))
    if dm == "ldavecs":
        ncols = 60
    else:
        ncols = 100
        
    print(len(l3))

    tp_chunks = pm[[pat_dict[p] for p in l3["tp"].tolist()]].compute()
    op_chunks = pm[[pat_dict[p] for p in l3["op"].tolist()]].compute()
    print(len(l3))

    # Split into chunks
    n_chunks = np.round(len(l3)/3000)
    tp_chunks = np.array_split(tp_chunks, n_chunks)
    op_chunks = np.array_split(op_chunks, n_chunks)

    print("Getting patent pair similarity")
    print("cosine")
    print(datetime.datetime.now())
    # Recursively lengthen the array of cosine distances
    cos_dis = np.array([])
    for i,j in zip(tp_chunks, op_chunks):
        cos_dis = np.hstack((cos_dis, np.hstack([np.diag(distance.cdist(i,j, metric = "cosine"))])))
        
    l3["sim_{0}".format(dm)] = 1-cos_dis
    fastparquet.write("DataStore/2018-07-P2/citation_pairs_{0}_sim_0716.parq".format(dm), l3, compression="GZIP")
    print("finished")
    print(datetime.datetime.now())
    del(l3)    
        

INFO:root:Loading matrix and dict
INFO:root:2018-07-16 13:41:40.220091
INFO:root:8070579
INFO:root:7134537
INFO:root:7134537
INFO:root:7134537
INFO:root:Getting patent pair similarity
INFO:root:cosine
INFO:root:2018-07-16 13:42:25.153565
INFO:root:finished
INFO:root:2018-07-16 13:55:38.412306
INFO:root:Loading matrix and dict
INFO:root:2018-07-16 13:55:38.414460
INFO:root:8070579
INFO:root:7134537
INFO:root:7134537
INFO:root:7134537
INFO:root:Getting patent pair similarity
INFO:root:cosine
INFO:root:2018-07-16 13:56:43.843546
INFO:root:finished
INFO:root:2018-07-16 14:15:41.195091


### 3. Merge data and get average per citing patent

In [3]:
cit = fastparquet.ParquetFile("DataStore/2018-07-P2/citation_pairs_samp_0716.parq").to_pandas()
csim = fastparquet.ParquetFile("DataStore/2018-07-P2/citation_pairs_ldavecs_sim_0716.parq").to_pandas()
r2 = fastparquet.ParquetFile("DataStore/2018-07-P2/citation_pairs_docvecs_sim_0716.parq").to_pandas()
csim = csim.merge(r2, how = "left", on = ["tp", "op"])
csim = csim.rename(columns={"tp": "cited", "op":"citing"})
del(r2)
cit = cit.merge(csim, how = "left", on = ["citing", "cited"])
del(csim)

pdf = fastparquet.ParquetFile("PatBib/inv_pat_us_7616_loc_unique.parq").to_pandas(["patent", "gyear", 
                                                                                   "inv_msa", "naics_name",
                                                                                  "primclass", "asgname"])
cit = cit.merge(pdf.add_prefix("cited_"), how = "left", left_on = "cited", right_on = "cited_patent")\
.drop("cited_patent",1)
cit = cit.merge(pdf.add_prefix("citing_"), how = "left", left_on = "citing", right_on = "citing_patent")\
.drop("citing_patent",1)
for c in ["inv_msa", "naics_name","primclass", "asgname"]:
    cit["{0}_match".format(c)] = np.nan
    cond = (cit["citing_{0}".format(c)].notnull() & cit["cited_{0}".format(c)].notnull())
    cit.loc[cond, "{0}_match".format(c)] = (cit.loc[cond, "citing_{0}".format(c)] \
                                            == cit.loc[cond, "cited_{0}".format(c)])
                
    cit = cit.drop(["citing_{0}".format(c), "cited_{0}".format(c)], 1)
del(pdf)

In [4]:
# Drop whatever is missing all key columns
print(len(cit))
cit = cit.dropna(how="any", subset=['sim_ldavecs', 'sim_docvecs',
        'inv_msa_match', 'naics_name_match', 'primclass_match',
       'asgname_match'])
print(len(cit))

8155647
5427823


In [8]:
# # Scale similarities
# # Don't scale
# # cit = fastparquet.ParquetFile("DataStore/2018-07-P2/citation_pairs_0716.parq").to_pandas()
# eps = 0.01
# dv_min = 0.7
# def scale_docvecs(x):
#     scaled = ((x+dv_min)/(1+dv_min))*(1-eps)+eps
#     return scaled
# def scale_ldavecs(x):
#     scaled = x*(1-eps)+eps
#     return scaled
# cit["sim_ldavecs"] = cit["sim_ldavecs"].apply(scale_ldavecs)
# cit["sim_docvecs"] = cit["sim_docvecs"].apply(scale_docvecs)

In [5]:
fastparquet.write("DataStore/2018-07-P2/citation_pairs_0716.parq", cit, compression="GZIP")

In [6]:
# 1. Get average number of citations each patent makes
cit = fastparquet.ParquetFile("DataStore/2018-07-P2/citation_pairs_0716.parq").to_pandas()
c2 = cit[["citing", "cited"]].groupby("citing").size().reset_index()
print(len(c2))
# 2. Get average inv_msa_match by cited patent
c3 = cit[["citing", "inv_msa_match", "naics_name_match", "primclass_match", "citing_gyear",
         "sim_ldavecs", "sim_docvecs"]].groupby("citing").mean().reset_index()

# 3. Merge with c2
c2 = c2.merge(c3, how = "left", on = "citing")
c2 = c2.rename(columns={0: "av_num_cited"})
print(len(c2))
# Drop any missing
c2 = c2.dropna(how="any")
print(len(c2))
# 4. Save
c2.to_pickle("DataStore/2018-07-P2/citing_summ_0716.pkl")
del(c2, c3)

1044956
1044956
1044956


### Average forward and backward citations by year

In [3]:
# 1. Get average number of citations each patent makes
cit = fastparquet.ParquetFile("DataStore/2018-07-P2/citation_pairs_0716.parq").to_pandas()

In [7]:

back_sim = cit[["citing", "citing_gyear",
         "sim_ldavecs", "sim_docvecs"]].groupby("citing").mean().reset_index()

back_sim = back_sim.groupby("citing_gyear").mean()

back_cite = cit[["citing", "cited"]].groupby("citing").size().reset_index()

In [12]:
# Amount of backward citations per patent: groupby citing, citing_gyear
c2 = cit[["citing", "cited"]].groupby("citing").size().reset_index()
print(len(c2))
# 2. Get average inv_msa_match by cited patent
c3 = cit[["citing", "citing_gyear",
         "sim_ldavecs", "sim_docvecs"]].groupby("citing").mean().reset_index()

# 3. Merge with c2
c2 = c2.merge(c3, how = "left", on = "citing")
c2 = c2.rename(columns={0: "av_num_back_cited"})
print(len(c2))
# Drop any missing
c2 = c2.dropna(how="any")
print(len(c2))
# Groupby gyear
back_cite = c2[["av_num_back_cited", "citing_gyear", "sim_docvecs"]].groupby("citing_gyear").mean()

1044956
1044956
1044956


In [13]:
# Amount of forward citations per patent: groupby cited, cited_gyear
c2 = cit[["citing", "cited"]].groupby("cited").size().reset_index()
print(len(c2))
# 2. Get average inv_msa_match by cited patent
c3 = cit[["cited", "cited_gyear",
         "sim_ldavecs", "sim_docvecs"]].groupby("cited").mean().reset_index()

# 3. Merge with c2
c2 = c2.merge(c3, how = "left", on = "cited")
c2 = c2.rename(columns={0: "av_num_forward_cited"})
print(len(c2))
# Drop any missing
c2 = c2.dropna(how="any")
print(len(c2))
# Groupby gyear
forward_cite = c2[["av_num_forward_cited", "cited_gyear", "sim_docvecs"]].groupby("cited_gyear").mean()

917410
917410
917410


In [14]:
back_cite

,av_num_back_cited,sim_docvecs
citing_gyear,,
1976.0,1.037037,0.334152
1977.0,1.155978,0.325670
1978.0,1.312015,0.319728
1979.0,1.429100,0.313846
1980.0,1.606755,0.316435
1981.0,1.720491,0.309975
1982.0,1.916502,0.310989
1983.0,2.051657,0.308526
1984.0,2.134491,0.306753


In [15]:
forward_cite

,av_num_forward_cited,sim_docvecs
cited_gyear,,
1976.0,2.435371,0.301853
1977.0,2.566477,0.302688
1978.0,2.629406,0.300890
1979.0,2.749282,0.300945
1980.0,2.819653,0.297423
1981.0,2.881755,0.297675
1982.0,3.041056,0.295772
1983.0,3.242157,0.295278
1984.0,3.371057,0.292490
